# 01 - Import & Explore DAIC-WOZ Labels (then Clean)

This notebook establishes the **labels foundation** for the trauma-informed AI pipeline. 
Before working with multimodal features (audio, video, text), we must import, validate, and lightly clean the ground-truth **PHQ-8 depression scores** from the AVEC 2017 challenge. 

## Objectives
- **Load** the DAIC-WOZ / AVEC 2017 training labels (participant_id + PHQ-8 scores). 
- **Inspect** structure, missing values, duplicates, and valid ranges. 
- **Visualize** label distributions (continuous PHQ-8 and binary depressed vs. non-depressed). 
- **Clean minimally** (drop duplicates, enforce ranges, standardize IDs). 
- **Save outputs**: 
 - A tiny shareable sample (for fast iteration). 
 - A full cleaned artifact (for modeling, ignored by git). 

## Why This Matters
Accurate labels are the backbone of supervised learning. By carefully auditing and documenting them first, we ensure that downstream feature engineering and modeling rest on a **clean, reproducible, and trustworthy foundation**.

---


In [ ]:
# --- bootstrap PYTHONPATH so repo utilities are importable ------------------------------
# Why: Jupyter often runs from the `notebooks/` folder, so `utils/` (at repo root) isn't on sys.path.
# This cell adds both ROOT and ROOT/utils to sys.path so `from utils.sanity ...` works reliably.
import sys, pathlib
CWD = pathlib.Path.cwd()
ROOT = CWD if (CWD / "utils").exists() else CWD.parent # handles when notebook lives in notebooks/
if str(ROOT) not in sys.path:
 sys.path.append(str(ROOT))
if str(ROOT / "utils") not in sys.path:
 sys.path.append(str(ROOT / "utils"))
print("ROOT:", ROOT)
print("In sys.path:", str(ROOT) in sys.path, str(ROOT/'utils') in sys.path)

In [ ]:
# --- environment sanity & project paths -------------------------------------------------
# Why: print python path/versions and set up canonical DATA folders used throughout the project.
from utils.sanity import sanity_env, setup_paths, set_seeds
sanity_env(pkgs=("pandas","numpy","matplotlib","seaborn","sklearn"))
ROOT, DATA, RAW, CLEAN, OUT = setup_paths()
set_seeds(42)
ROOT, DATA, RAW, CLEAN, OUT

## Step 1 - Discover raw files

Resolve the repository root once, define canonical `DATA/RAW/CLEAN/PROCESSED` paths,
and list likely data files to verify locations and extensions before import.


## 1A - Find repo root and define canonical paths

In [ ]:
import sys
from pathlib import Path


def _add_src_to_sys_path():
 here = Path.cwd().resolve()
 for cand in [here, *here.parents]:
 src_dir = cand / "src"
 if src_dir.exists():
 sys.path.insert(0, str(src_dir))
 return str(src_dir)
 return None


print("Added to PYTHONPATH:", _add_src_to_sys_path())

from config_paths import ROOT, DATA, RAW, CLEAN, PROCESSED

print("CWD:", Path.cwd().resolve())
print("ROOT:", ROOT)
print("RAW:", RAW, "| exists:", RAW.exists(), "| is_dir:", RAW.is_dir())
print("CLEAN:", CLEAN, "| exists:", CLEAN.exists())
print("PROCESSED:", PROCESSED, "| exists:", PROCESSED.exists())





## Step 2 - Load the Depression AVEC 2017 training labels
We expect a file named like `train_split_Depression_AVEC2017.csv` containing **participant IDs**
and **PHQ 8** totals. Adjust `labels_rel` below if your path differs.


---
## 1B - Discover raw files

In [ ]:
from config_paths import list_files

TOP_N = 30
raw_list = list_files(RAW)
print(f"Found {len(raw_list)} candidate files under {RAW}")
for rel, mb in raw_list[:TOP_N]:
 print(f"{str(rel):70s} {mb:6.2f} MB")



## Step 3 - Normalize column names; choose ID & label columns

In [ ]:
# Normalize: strip, lowercase, spaces->underscores for consistent referencing
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
print("Normalized columns:", df.columns.tolist())

# For AVEC 2017 labels we expect these names; override if different:
ID_COL = "participant_id"
LABEL_COL = "phq8_score"

assert ID_COL in df.columns, f"{ID_COL} not in columns: {df.columns.tolist()}"
assert LABEL_COL in df.columns, f"{LABEL_COL} not in columns: {df.columns.tolist()}"

# Standardize types: IDs as strings; coerce label to numeric
df[ID_COL] = df[ID_COL].astype(str).str.strip()
df[LABEL_COL] = pd.to_numeric(df[LABEL_COL], errors="coerce")

print(f"Using ID_COL={ID_COL}, LABEL_COL={LABEL_COL}")
df[[ID_COL, LABEL_COL]].head(5)

## Step 4 - Overview & nulls audit

In [ ]:
# --- Quick overview and null audit -------------------------------------------
# Why: understand the shape of the dataset and check for missing values
# across critical columns (IDs + PHQ8 scores).

from utils.sanity import data_overview

data_overview(df[[ID_COL, LABEL_COL]])


## Step 5 - Label distribution & integrity checks

In [ ]:
# --- Label distribution and integrity checks ---------------------------------
# Why: 
# - visualize the PHQ8 score distribution (class imbalance, expected range 0-24)
# - check for duplicate participant IDs
# - confirm all scores fall in the valid range

from utils.sanity import label_balance, check_integrity

label_balance(df, label_col=LABEL_COL, binary_col="phq8_binary")

check_integrity(
 df,
 id_col=ID_COL,
 label_col=LABEL_COL,
 label_range=(0, 24) # valid PHQ8 score range
)


## Step 6 - Minimal cleaning (documented, reversible)

In [ ]:
# --- Minimal cleaning --------------------------------------------------------
# Why:
# - Drop rows missing ID or PHQ8 score (can't be used downstream).
# - Drop any duplicate IDs (should be 0, but defensive coding is good).
# - Keep only scores in the valid PHQ8 range (0-24).

clean = (
 df.dropna(subset=[ID_COL, LABEL_COL]) # drop rows with null ID or score
 .drop_duplicates(subset=[ID_COL]) # keep first occurrence per participant
 .query(f"{LABEL_COL} >= 0 & {LABEL_COL} <= 24") # enforce valid score range
 .copy()
)

# standardize ID again to string + strip whitespace
clean[ID_COL] = clean[ID_COL].astype(str).str.strip()

print("Before -> After:", len(df), "->", len(clean))
clean.head()


## Step 7 - Save artifacts (tiny sample + full cleaned)

In [ ]:
from utils.sanity import save_checkpoint

# Tiny sample (<=200 rows) - safe to version if you want
_ = save_checkpoint(clean, OUT / "eda_sample.parquet", n=min(200, len(clean)))

# Full cleaned labels - ignored by git (lives in data/cleaned/)
CLEAN.mkdir(parents=True, exist_ok=True)
full_path = CLEAN / "labels_clean.parquet"
clean.to_parquet(full_path, index=False)
print("Saved:", full_path)


---
## Step 7 - Save Artifacts (Tiny Sample + Full Cleaned)
We checkpoint the cleaned dataset into two outputs:

- **Tiny sample** (`eda_sample.parquet`) a lightweight 200-row slice for quick testing and sharing. 
- **Full cleaned labels** (`labels_clean.parquet`) the authoritative dataset, ignored by git to prevent large file commits. 

**Key Point:** Saving both a tiny and full version makes development easier (fast iterations) while preserving the complete, cleaned dataset for modeling.

---

## Appendix - Working Data Dictionary (Updated as I Go)

- **participant_id** *(string)* - normalized unique subject identifier. 
- **phq8_score** *(int, 0-24)* - PHQ-8 depression severity total score. 
- **phq8_binary** *(int, 0-1)* - derived label (0 = non-depressed, 1 = depressed). 
- **gender** *(string)* - participant gender (encoding not yet confirmed). 
- **phq8_nointerest** *(int, 0-3)* - item 1 (loss of interest/pleasure). 
- **phq8_depressed** *(int, 0-3)* - item 2 (feeling down/depressed). 
- **phq8_sleep** *(int, 0-3)* - item 3 (sleep disturbance). 
- **phq8_tired** *(int, 0-3)* - item 4 (fatigue/low energy). 
- **phq8_appetite** *(int, 0-3)* - item 5 (appetite changes). 
- **phq8_failure** *(int, 0-3)* - item 6 (feelings of failure/self-worth). 
- **phq8_concentrating** *(int, 0-3)* - item 7 (trouble concentrating). 
- **phq8_moving** *(int, 0-3)* - item 8 (psychomotor agitation/retardation). 

**Provenance:** Source file = `train_split_Depression_AVEC2017.csv` (DAIC-WOZ / AVEC 2017 depression dataset). 

**Assumptions:** 
- Valid PHQ-8 score range is 0-24. 
- No duplicate participant IDs. 
- `phq8_binary` derived using clinical cutoff (e.g., 10). 

**Artifacts generated:** 
- `outputs/eda_sample.parquet` shareable sample 
- `data/cleaned/labels_clean.parquet` full cleaned labels table 

**Next:** 
- Confirm gender encoding. 
- Optionally join with demographics/transcripts on `participant_id`. 

---



## Closing Summary
In this notebook we successfully:

1. Imported and verified raw DAIC-WOZ depression label files. 
2. Normalized column names, standardized IDs, and enforced valid ranges. 
3. Audited dataset completeness and confirmed no missing values in critical fields. 
4. Visualized PHQ-8 distributions, noting a class imbalance (~72% vs. ~28%). 
5. Performed minimal but effective cleaning to remove duplicates and invalid rows. 
6. Saved both lightweight and full cleaned artifacts for downstream use. 

This establishes a **clean, reproducible foundation** for modeling. The outputs from this notebook will directly feed into feature engineering, baseline models, and future multimodal fusion experiments.

---

**Next Steps ** Proceed to [02_model_baselines.ipynb](./02_model_baselines.ipynb), 
where we establish baseline classifiers (Dummy vs. Logistic) and address the class imbalance identified here.

---